In [43]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import seaborn as sns

from pathlib import Path

In [44]:
fp_original = "Data/LS Biomarkers Timeseries.csv"
fp_long = "Data/LS_Biomarkers_Long.csv"
fp_wide = "Data/LS_Biomarkers_Wide.csv"

In [45]:
df_BioDairyLS  = pd.read_csv(fp_original)
df_BioDairyLS  

,Date collected,Units,Good Range,5/1/1993,11/27/1996,5/8/1997,4/15/1998,4/1/1999,10/6/1999,5/7/2001,...,2/5/2025,2/12/2025,2/28/2025,5/20/2025,6/13/2025,Low,High,Average,Number,Unnamed: 341
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,YFH/UCSD,UCSD,UCSD,UCSD,UCSD,of tests since '07,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,201.0,...,178,179,179.6,182.5,180.8,166.1,202.0,185.6,322.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Blood Tests,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Butyrate,mg/mL,0.8-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,NaN,NaN,NaN,0.3,4.3,1.3,184.0,NaN
234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create Long Format

## Helper Functions

In [46]:
def is_blank(x):
    return (x is None) or (pd.isna(x)) or (str(x).strip() == "")

def find_col(name_like, default=None):
    for c in cols:
        if name_like in str(c).strip().lower():
            return c
    return default

def find_exact(label):
    # return exact match if present; else case-insensitive match
    for c in cols:
        if c == label:
            return c
    low = label.lower()
    for c in cols:
        if str(c).strip().lower() == low:
            return c
    return None

def uniq_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x is None: 
            continue
        s = str(x).strip()
        if not s or s in seen:
            continue
        seen.add(s)
        out.append(s)
    return out

def clean_label(x):
    if is_blank(x):
        return None
    # remove non-breaking spaces and trim outer whitespace
    return str(x).replace("\u00a0", " ").strip()

# Mapping Dictionaries for Columns

In [47]:
# Mapping For biomarkeers to panels
dict_biomarkerToPanels = {
                            # Vitals
                            "Weight": "Vitals",

                            # LIPID Panel
                            "237": "LIPID Panel",
                            'HDL (High Density Lipoprotein) "good"': "LIPID Panel",
                            "VLDL Cholesterol": "LIPID Panel",
                            "IDL Chloresterol": "LIPID Panel",
                            "Sum Total Cholesterol": "LIPID Panel",
                            "Triglycerides": "LIPID Panel",
                            "UCSD LDL Particle # by NMR": "LIPID Panel",
                            "UCSD HDL Particle #": "LIPID Panel",
                            "UCSD Large VLDL Particle #": "LIPID Panel",
                            "UCSD Small LDL Particle #": "LIPID Panel",
                            "UCSD Large HDL Particle #": "LIPID Panel",
                            "UCSD VLDL Particle Size": "LIPID Panel",
                            "UCSD LDL Particle Size": "LIPID Panel",
                            "UCSD HDL Particle Size": "LIPID Panel",
                            "Real-LDL Size Pattern": "LIPID Panel",
                            "TG/HDL": "LIPID Panel",
                            "Chol/HDL": "LIPID Panel",
                            "Lp(a) Chloresterol": "LIPID Panel",
                            "APO A1": "LIPID Panel",
                            "APOLIPOPROTEIN-(B100)": "LIPID Panel",
                            "Ratio APO B100/AI": "LIPID Panel",

                            # Comprehensive Metabolic Panel – Electrolytes
                            "Sodium": "Comprehensive Metabolic Panel",
                            "Potassium": "Comprehensive Metabolic Panel",
                            "Chloride": "Comprehensive Metabolic Panel",
                            "Calcium": "Comprehensive Metabolic Panel",
                            "Phosporus": "Comprehensive Metabolic Panel",
                            "Magnesium": "Comprehensive Metabolic Panel",
                            "Zinc": "Comprehensive Metabolic Panel",
                            "Tot CO2": "Comprehensive Metabolic Panel",
                            "Bicarbonate": "Comprehensive Metabolic Panel",
                            "Anion Gap": "Comprehensive Metabolic Panel",
                            
                            # Comprehensive Metabolic Panel – remainder
                            "Total Protein": "Comprehensive Metabolic Panel",
                            "Albumin": "Comprehensive Metabolic Panel",
                            "Globulin": "Comprehensive Metabolic Panel",
                            "A/G ratio": "Comprehensive Metabolic Panel",
                            "GGTP": "Comprehensive Metabolic Panel",
                            "LDH": "Comprehensive Metabolic Panel",
                            "ALT": "Comprehensive Metabolic Panel",
                            "AST": "Comprehensive Metabolic Panel",
                            "AST/ALT ratio": "Comprehensive Metabolic Panel",
                            "Alkaline phosphatase": "Comprehensive Metabolic Panel",
                            "Total Bilirubin": "Comprehensive Metabolic Panel",
                            "Direct Bilirubin": "Comprehensive Metabolic Panel",

                            # Kidney
                            "Blood Urea Nitrogen (BUN)": "Kidney",
                            "Creatinine": "Kidney",
                            "BUN/Creatinine Ratio": "Kidney",
                            "Uric Acid": "Kidney",

                            # Glucose/Insulin Panel
                            "Fasting Glucose": "Glucose/Insulin Panel",
                            "Insulin": "Glucose/Insulin Panel",
                            "Hemoglobin A1C": "Glucose/Insulin Panel",
                            "HOMA-IR (insulin resistance)": "Glucose/Insulin Panel",

                            # Hormone Panel
                            "Free Testosterone": "Hormone Panel",
                            "Testosterone % Free": "Hormone Panel",
                            "Total Testosterone": "Hormone Panel",
                            "Sex Hormone Binding Globulin (SHBG)": "Hormone Panel",
                            "Estradiol": "Hormone Panel",
                            "DHEA-blood": "Hormone Panel",
                            "Cortisol Random": "Hormone Panel",
                            "TSH": "Hormone Panel",
                            "Total T3": "Hormone Panel",
                            "Thyroxine (T4)": "Hormone Panel",

                            # Cancer Panel
                            "PSA Screen": "Cancer Panel",
                            "CA-19-9 (Pancreatic Cancer)": "Cancer Panel",
                            "CEA": "Cancer Panel",

                            # Iron Panel
                            "Iron": "Iron Panel",
                            "Ferritin": "Iron Panel",
                            "Fibrinogen": "Iron Panel",
                            "TIBC": "Iron Panel",
                            "UIBC": "Iron Panel",
                            "Iron saturation": "Iron Panel",

                            # CBC – Red blood cell details
                            "RBC-": "CBC: Red Blood Cell Details",
                            "HGB-": "CBC: Red Blood Cell Details",
                            "HCT": "CBC: Red Blood Cell Details",
                            "PLT-": "CBC: Red Blood Cell Details",
                            "MCV-": "CBC: Red Blood Cell Details",
                            "MCH": "CBC: Red Blood Cell Details",
                            "MCHC-": "CBC: Red Blood Cell Details",
                            "RDW": "CBC: Red Blood Cell Details",
                            "MPV": "CBC: Red Blood Cell Details",

                            # CBC – White blood cell details
                            "WBC-": "CBC: White Blood Cell Details",
                            "NEU %": "CBC: White Blood Cell Details",
                            "LYM %": "CBC: White Blood Cell Details",
                            "EOS %": "CBC: White Blood Cell Details",
                            "MONO %": "CBC: White Blood Cell Details",
                            "BASO %": "CBC: White Blood Cell Details",
                            "Neutrophil Count": "CBC: White Blood Cell Details",
                            "Lymphocyte Count": "CBC: White Blood Cell Details",
                            "Monocyte Count": "CBC: White Blood Cell Details",
                            "Esoinophil Count": "CBC: White Blood Cell Details",
                            "Basophil Count": "CBC: White Blood Cell Details",

                            # Inflammation Panel
                            "High Sensitivity CRP (mg/L)": "Inflammation Panel",
                            "Lp-PLA2": "Inflammation Panel",
                            "IgG": "Inflammation Panel",
                            "IgA": "Inflammation Panel",
                            "IgM": "Inflammation Panel",

                            # Vitamins, Antioxidants, Amino Acid Panel
                            "Homocysteine": "Vitamins, Antioxidants, Amino Acid Panel",
                            "CoQ10": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Selenium": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin E Alpha-Tocopherol": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin E Gamma-Tocopherol": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Alpha Lipoic Acid": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin D, 25-Hydroxy": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin B12": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Niacin (Vit B3)": "Vitamins, Antioxidants, Amino Acid Panel",

                            # Stool tests
                            "pH": "Stool tests",

                            # Cultured Bacteria Growth, Expected/Beneficial flora
                            "Bifidobacerium family": "Expected/Beneficial flora",
                            "E. coli spp.": "Expected/Beneficial flora",
                            "Lactobacillus spp": "Expected/Beneficial flora",
                            "Enterococcus spp.": "Expected/Beneficial flora",
                            "Bacteroides fraglis group": "Expected/Beneficial flora",
                            "Clostridium spp": "Expected/Beneficial flora",
                            "Bacteroides family": "Expected/Beneficial flora",

                            # Cultured Bacteria Growth, Commensal (Imbalanced) flora
                            "Alpha hemolytic strep": "Commensal (Imbalanced) flora",
                            "Hemolytic Escherichia coli": "Commensal (Imbalanced) flora",
                            "Pseudomonas chlororaphis group": "Commensal (Imbalanced) flora",
                            "Gamma Hemolytic Streptococcus": "Commensal (Imbalanced) flora",
                            "Hafnia alvei": "Commensal (Imbalanced) flora",
                            "Mucoid Escherichia coli": "Commensal (Imbalanced) flora",
                            "Beta strep, group B, strep. Agalactiae": "Commensal (Imbalanced) flora",
                            "Klebsiella oxytoca": "Commensal (Imbalanced) flora",
                            "Bacillus spp, pumilus": "Commensal (Imbalanced) flora",
                            "Comamonas testosteroni": "Commensal (Imbalanced) flora",
                            "Streptococcus anginosus": "Commensal (Imbalanced) flora",
                            "Cellulosimicrobium cellulans/funkeil/marinum": "Commensal (Imbalanced) flora",
                            "Lactococcus lactis": "Commensal (Imbalanced) flora",
                            "Staphylococcus aureus": "Commensal (Imbalanced) flora",
                            "Streptococcus salivarius": "Commensal (Imbalanced) flora",
                            "Streptococcus parasanguinis": "Commensal (Imbalanced) flora",
                            "Pseudomonas mendocina ": "Commensal (Imbalanced) flora",
                            "Actinomyces turicensis": "Commensal (Imbalanced) flora",
                            "Lysinibacillus spp": "Commensal (Imbalanced) flora",
                            "Streptococcus mutans": "Commensal (Imbalanced) flora",
                            "Streptococcus australis": "Commensal (Imbalanced) flora",
                            "Enterobacter asburiae": "Commensal (Imbalanced) flora",
                            "Pseudomonas stutzeri": "Commensal (Imbalanced) flora",
                            "Kocuria spp, salsicia, rhizophila, kristinae": "Commensal (Imbalanced) flora",
                            "Rothia dentocariosa/mucliaginosa": "Commensal (Imbalanced) flora",
                            "Pantoea spp": "Commensal (Imbalanced) flora",
                            "Pseudomonas citronellolis": "Commensal (Imbalanced) flora",
                            "Streptococcus mitis/oralis/cristatus": "Commensal (Imbalanced) flora",
                            "Staphylococcus haemolyticus": "Commensal (Imbalanced) flora",
                            "Serratia marcescens": "Commensal (Imbalanced) flora",
                            "Staphylococcus lugdunensis": "Commensal (Imbalanced) flora",
                            "Staphylococcus epidermidis": "Commensal (Imbalanced) flora",
                            "Actinomyces neuii": "Commensal (Imbalanced) flora",
                            "Streptomyces": "Commensal (Imbalanced) flora",
                            "Bacillus licheniformis": "Commensal (Imbalanced) flora",
                            "Corynebacterium falsenii": "Commensal (Imbalanced) flora",

                            # Dysbiotic bacteria
                            "Morganellla morganii": "Dysbiotic bacteria",
                            "Providencia stuartii": "Dysbiotic bacteria",
                            "Klebsiella pneumonia": "Dysbiotic bacteria",
                            "Providencia rettgeri": "Dysbiotic bacteria",
                            "Enterobacter cloacae": "Dysbiotic bacteria",
                            "Pseudomonas aeruginosa": "Dysbiotic bacteria",
                            "Bacillus cereus": "Dysbiotic bacteria",
                            "Citrobacter freundii complex": "Dysbiotic bacteria",
                            "Arcobacter butzieri": "Dysbiotic bacteria",

                            # Yeast
                            "Candida parapsilosis": "Yeast",
                            "Rhodotorula glutinis/mucilaginosa": "Yeast",
                            "Candida krusei/inconspicua/lambica": "Yeast",
                            "Saccharomyces cerevisiae/boulardi": "Yeast",
                            "Geotrichum spp.": "Yeast",
                            "Candida pararugosa": "Yeast",
                            "Candida lusitaniae": "Yeast",
                            "Candida zeylanoides": "Yeast",
                            "Trichosporon asahii": "Yeast",
                            "Trichosporon ovoides": "Yeast",
                            "Trichosporon montevideense": "Yeast",
                            "Exophiala dermatitidis": "Yeast",
                            "Cryptococcus spp, not neoformans": "Yeast",
                            "Cryptococcus diffuens": "Yeast",
                            "Candida orthopsilosis": "Yeast",

                            # Gut Inflammation
                            "Stool Calprotectin UCSD": "Gut Inflammation",
                            "Stool Calprotectin YFH": "Gut Inflammation",
                            "Stool Lysozyme": "Gut Inflammation",
                            "Stool Lactoferrin": "Gut Inflammation",
                            "Stool SIgA": "Gut Inflammation",

                             # Short Chain Fatty Acids
                            "% SCFA Acetate": "Short Chain Fatty Acids",
                            "% SCFA Propionate": "Short Chain Fatty Acids",
                            "% SCFA Valerate": "Short Chain Fatty Acids",
                            "% SCFA Butyrate": "Short Chain Fatty Acids",
                            "Total SCFA": "Short Chain Fatty Acids",
                            "Butyrate": "Short Chain Fatty Acids"

}

# Mapping for units to biomarkers
dict_unitsToBiomarkers = {
                            # Vitals
                            "Weight": "lbs",

                            # LIPID Panel
                            "237": "mg/dL",
                            'HDL (High Density Lipoprotein) "good"': "mg/dL",
                            "VLDL Cholesterol": "mg/dL",
                            "IDL Chloresterol": None,
                            "Sum Total Cholesterol": "mg/dL",
                            "Triglycerides": "mg/dL",
                            "UCSD LDL Particle # by NMR": "nmol/L",
                            "UCSD HDL Particle #": "μmol/L",
                            "UCSD Large VLDL Particle #": "nmol/L",
                            "UCSD Small LDL Particle #": "nmol/L",
                            "UCSD Large HDL Particle #": "μmol/L",
                            "UCSD VLDL Particle Size": "nm",
                            "UCSD LDL Particle Size": "nm",
                            "UCSD HDL Particle Size": "nm",
                            "Real-LDL Size Pattern": None,
                            "TG/HDL": None,
                            "Chol/HDL": None,
                            "Lp(a) Chloresterol": None,
                            "APO A1": "mg/dL",
                            "APOLIPOPROTEIN-(B100)": "mg/dL",
                            "Ratio APO B100/AI": None,

                            # Comprehensive Metabolic Panel – Electrolytes
                            "Sodium": "mmol/L",
                            "Potassium": "mmol/L",
                            "Chloride": "mmol/L",
                            "Calcium": "mg/dL",
                            "Phosporus": "mg/dL",
                            "Magnesium": "mg/dL",
                            "Zinc": "µg/dL",
                            "Tot CO2": None,
                            "Bicarbonate": "mmol/L",
                            "Anion Gap": "mmol/L",

                            # Comprehensive Metabolic Panel – remainder
                            "Total Protein": "g/dL",
                            "Albumin": "g/dL",
                            "Globulin": None,
                            "A/G ratio": None,
                            "GGTP": "U/L",
                            "LDH": "U/L",
                            "ALT": "U/L",
                            "AST": "U/L",
                            "AST/ALT ratio": None,
                            "Alkaline phosphatase": "U/L",
                            "Total Bilirubin": "mg/dL",
                            "Direct Bilirubin": None,

                            # Kidney
                            "Blood Urea Nitrogen (BUN)": "mg/dL",
                            "Creatinine": "mg/dL",
                            "BUN/Creatinine Ratio": None,
                            "Uric Acid": None,

                            # Glucose/Insulin Panel
                            "Fasting Glucose": "mg/dL",
                            "Insulin": "µIU/mL",
                            "Hemoglobin A1C": "%",
                            "HOMA-IR (insulin resistance)": None,

                            # Hormone Panel
                            "Free Testosterone": "pg/mL",
                            "Testosterone % Free": "%",
                            "Total Testosterone": "ng/mL",
                            "Sex Hormone Binding Globulin (SHBG)": "nmol/L",
                            "Estradiol": "pg/mL",
                            "DHEA-blood": "µg/dL",
                            "Cortisol Random": None,
                            "TSH": "µIU/mL",
                            "Total T3": "ng/mL",
                            "Thyroxine (T4)": "µg/dL",

                            # Cancer Panel
                            "PSA Screen": None,
                            "CA-19-9 (Pancreatic Cancer)": "U/mL",
                            "CEA": "ng/mL",

                            # Iron Panel
                            "Iron": "µg/dL",
                            "Ferritin": "ng/mL",
                            "Fibrinogen": None,
                            "TIBC": "µg/dL",
                            "UIBC": "µg/dL",
                            "Iron saturation": "%",

                            # Complete Blood Count (CBC) Panel – Red Blood Cell Details
                            "RBC-": "10**6/mL",
                            "HGB-": "g/dL",
                            "HCT": "%",
                            "PLT-": "10**3/mL",
                            "MCV-": "fL",
                            "MCH": "pg/cell",
                            "MCHC-": "g/dL",
                            "RDW": "%",
                            "MPV": "fL",

                            # Complete Blood Count (CBC) Panel  – White Blood Cell Details
                            "WBC-": "10**3/mL",
                            "NEU %": "%",
                            "LYM %": "%",
                            "EOS %": "%",
                            "MONO %": "%",
                            "BASO %": "%",
                            "Neutrophil Count": "10**3/mL",
                            "Lymphocyte Count": "10**3/mL",
                            "Monocyte Count": "10**3/mL",
                            "Esoinophil Count": "10**3/mL",
                            "Basophil Count": "10**3/mL",

                            # Inflammation Panel
                            "High Sensitivity CRP (mg/L)": "mg/L",
                            "Lp-PLA2": None,
                            "IgG": None,
                            "IgA": None,
                            "IgM": None,

                            # Vitamins, Antioxidants, Amino Acid Panel
                            "Homocysteine": "µmol/L",
                            "CoQ10": None,
                            "Selenium": None,
                            "Vitamin E Alpha-Tocopherol": "mg/L",
                            "Vitamin E Gamma-Tocopherol": "mg/L",
                            "Alpha Lipoic Acid": None,
                            "Vitamin D, 25-Hydroxy": "ng/mL",
                            "Vitamin B12": "pg/mL",
                            "Niacin (Vit B3)": None,

                            # Stool Tests
                            "pH": None,

                            # Cultured Bacteria Growth - Expected/Beneficial flora
                            "Bifidobacerium family": None,
                            "E. coli spp.": None,
                            "Lactobacillus spp": None,
                            "Enterococcus spp.": None,
                            "Bacteroides fraglis group": None,
                            "Clostridium spp": None,
                            "Bacteroides family": None,

                            # Cultured Bacteria Growth - Commensal (Imbalanced) flora 
                            "Alpha hemolytic strep": None,
                            "Hemolytic Escherichia coli": None,
                            "Pseudomonas chlororaphis group": None,
                            "Gamma Hemolytic Streptococcus": None,
                            "Hafnia alvei": None,
                            "Mucoid Escherichia coli": None,
                            "Beta strep, group B, strep. Agalactiae": None,
                            "Klebsiella oxytoca": None,
                            "Bacillus spp, pumilus": None,
                            "Comamonas testosteroni": None,
                            "Streptococcus anginosus": None,
                            "Cellulosimicrobium cellulans/funkeil/marinum": None,
                            "Lactococcus lactis": None,
                            "Staphylococcus aureus": None,
                            "Streptococcus salivarius": None,
                            "Streptococcus parasanguinis": None,
                            "Pseudomonas mendocina ": None,
                            "Actinomyces turicensis": None,
                            "Lysinibacillus spp": None,
                            "Streptococcus mutans": None,
                            "Streptococcus australis": None,
                            "Enterobacter asburiae": None,
                            "Pseudomonas stutzeri": None,
                            "Kocuria spp, salsicia, rhizophila, kristinae": None,
                            "Rothia dentocariosa/mucliaginosa": None,
                            "Pantoea spp": None,
                            "Pseudomonas citronellolis": None,
                            "Streptococcus mitis/oralis/cristatus": None,
                            "Staphylococcus haemolyticus": None,
                            "Serratia marcescens": None,
                            "Staphylococcus lugdunensis": None,
                            "Staphylococcus epidermidis": None,
                            "Actinomyces neuii": None,
                            "Streptomyces": None,
                            "Bacillus licheniformis": None,
                            "Corynebacterium falsenii": None,

                            # Dysbiotic bacteria — no units
                            "Morganellla morganii": None,
                            "Providencia stuartii": None,
                            "Klebsiella pneumonia": None,
                            "Providencia rettgeri": None,
                            "Enterobacter cloacae": None,
                            "Pseudomonas aeruginosa": None,
                            "Bacillus cereus": None,
                            "Citrobacter freundii complex": None,
                            "Arcobacter butzieri": None,

                            # Yeast — no units
                            "Candida parapsilosis": None,
                            "Rhodotorula glutinis/mucilaginosa": None,
                            "Candida krusei/inconspicua/lambica": None,
                            "Saccharomyces cerevisiae/boulardi": None,
                            "Geotrichum spp.": None,
                            "Candida pararugosa": None,
                            "Candida lusitaniae": None,
                            "Candida zeylanoides": None,
                            "Trichosporon asahii": None,
                            "Trichosporon ovoides": None,
                            "Trichosporon montevideense": None,
                            "Exophiala dermatitidis": None,
                            "Cryptococcus spp, not neoformans": None,
                            "Cryptococcus diffuens": None,
                            "Candida orthopsilosis": None,

                            # Gut Inflammation
                            "Stool Calprotectin UCSD": "µg/g",
                            "Stool Calprotectin YFH": None,
                            "Stool Lysozyme": None,
                            "Stool Lactoferrin": None,
                            "Stool SIgA": None,

                            # Short Chain Fatty Acids
                            "% SCFA Acetate": "%",
                            "% SCFA Propionate": "%",
                            "% SCFA Valerate": "%",
                            "% SCFA Butyrate": "%",
                            "Total SCFA": "mg/mL",
                            "Butyrate": "mg/mL"

}

# Mapping for ranges to biomarkers
dict_biomarkersToRanges = {
                                # Vitals
                                "Weight": None,

                                # LIPID Panel
                                "237": "<129",
                                'HDL (High Density Lipoprotein) "good"': "40 to 59",
                                "VLDL Cholesterol": "<30",
                                "IDL Chloresterol": "<20",
                                "Sum Total Cholesterol": "<199",
                                "Triglycerides": "30 to 159",
                                "UCSD LDL Particle # by NMR": "<1135",
                                "UCSD HDL Particle #": ">33",
                                "UCSD Large VLDL Particle #": "<2.7",
                                "UCSD Small LDL Particle #": "<634",
                                "UCSD Large HDL Particle #": ">4.2",
                                "UCSD VLDL Particle Size": "<46.7",
                                "UCSD LDL Particle Size": ">20.7",
                                "UCSD HDL Particle Size": ">8.9",
                                "Real-LDL Size Pattern": "A",
                                "TG/HDL": "<2",
                                "Chol/HDL": "<4.5",
                                "Lp(a) Chloresterol": "<10",
                                "APO A1": "94 to 178",
                                "APOLIPOPROTEIN-(B100)": "55 to 140",
                                "Ratio APO B100/AI": None,

                                # Comprehensive Metabolic Panel – Electrolytes
                                "Sodium": "136 to 145",
                                "Potassium": "3.5 to 5.1",
                                "Chloride": "98 to 107",
                                "Calcium": "8.5 to 10.6",
                                "Phosporus": "2.7 to 4.5",
                                "Magnesium": "1.6 to 2.4",
                                "Zinc": "56-134",
                                "Tot CO2": "23-29",
                                "Bicarbonate": "22 to 29",
                                "Anion Gap": "7 to 15",

                                # Comprehensive Metabolic Panel – remainder
                                "Total Protein": "6 to 8",
                                "Albumin": "3.5 to 5.2",
                                "Globulin": "2.0-4.5",
                                "A/G ratio": "1.1-2.3",
                                "GGTP": "<60",
                                "LDH": "135 to 225",
                                "ALT": "0 to 41",
                                "AST": "0-40",
                                "AST/ALT ratio": None,
                                "Alkaline phosphatase": "40 to 120",
                                "Total Bilirubin": "<1.2",
                                "Direct Bilirubin": "<0.2",

                                # Kidney
                                "Blood Urea Nitrogen (BUN)": "8 to 23",
                                "Creatinine": "0.67 to 1.17",
                                "BUN/Creatinine Ratio": "10 to 20",
                                "Uric Acid": "4.0-8.5",

                                # Glucose/Insulin Panel
                                "Fasting Glucose": "70 to 99",
                                "Insulin": "3 to 19",
                                "Hemoglobin A1C": "4.8 to 5.8",
                                "HOMA-IR (insulin resistance)": "<2.8",

                                # Hormone Panel
                                "Free Testosterone": "47-244",
                                "Testosterone % Free": None,
                                "Total Testosterone": "2.8-8",
                                "Sex Hormone Binding Globulin (SHBG)": "15-100",
                                "Estradiol": "<43",
                                "DHEA-blood": "34 to 249",
                                "Cortisol Random": "6 to 23",
                                "TSH": "0.27 to 4.2",
                                "Total T3": "0.8 to 2.0",
                                "Thyroxine (T4)": "4.5 to 10.9",

                                # Cancer Panel
                                "PSA Screen": "<3.99",
                                "CA-19-9 (Pancreatic Cancer)": "30 to 42",
                                "CEA": "<3.8",

                                # Iron Panel
                                "Iron": "59 to 158",
                                "Ferritin": "30 to 400",
                                "Fibrinogen": "<460",
                                "TIBC": "148 to 506",
                                "UIBC": "112 to 346",
                                "Iron saturation": "?",

                                # Complete Blood Count (CBC) Panel – Red Blood Cell Details
                                "RBC-": "4.6 to 6.1",
                                "HGB-": "13.7 to 17.5",
                                "HCT": "40 to 50",
                                "PLT-": "140 to 370",
                                "MCV-": "79 to 95",
                                "MCH": "26 to 32",
                                "MCHC-": "32 to 36",
                                "RDW": "12 to 14",
                                "MPV": "9.4-12.4",

                                # Complete Blood Count (CBC) Panel  – White Blood Cell Details
                                "WBC-": "4 to 10",
                                "NEU %": "41 to 80",
                                "LYM %": "19 to 53",
                                "EOS %": "1 to 7",
                                "MONO %": "5 to 12",
                                "BASO %": "0 to 2",
                                "Neutrophil Count": "1.6 to 7",
                                "Lymphocyte Count": "0.8 to 3.1",
                                "Monocyte Count": "0.2 to 0.8",
                                "Esoinophil Count": "0.1 to 0.5",
                                "Basophil Count": "<0.1",

                                # Inflammation Panel
                                "High Sensitivity CRP (mg/L)": "<4.9",
                                "Lp-PLA2": "<200",
                                "IgG": "700-1600",
                                "IgA": "70-400",
                                "IgM": "40-230",

                                # Vitamins, Antioxidants, Amino Acid Panel
                                "Homocysteine": "0 to 14",
                                "CoQ10": "0.44-1.64",
                                "Selenium": None,
                                "Vitamin E Alpha-Tocopherol": "5.5-18",
                                "Vitamin E Gamma-Tocopherol": "0-6",
                                "Alpha Lipoic Acid": None,
                                "Vitamin D, 25-Hydroxy": "30-80",
                                "Vitamin B12": "232 to 1245",
                                "Niacin (Vit B3)": None,

                                # Stool Tests
                                "pH": "5.8 to 7",

                                # Cultured Bacteria Growth - Expected/Beneficial flora
                                "Bifidobacerium family": None,
                                "E. coli spp.": None,
                                "Lactobacillus spp": None,
                                "Enterococcus spp.": None,
                                "Bacteroides fraglis group": None,
                                "Clostridium spp": None,
                                "Bacteroides family": None,

                                # Cultured Bacteria Growth - Commensal (Imbalanced) flora 
                                "Alpha hemolytic strep": None,
                                "Hemolytic Escherichia coli": None,
                                "Pseudomonas chlororaphis group": None,
                                "Gamma Hemolytic Streptococcus": None,
                                "Hafnia alvei": None,
                                "Mucoid Escherichia coli": None,
                                "Beta strep, group B, strep. Agalactiae": None,
                                "Klebsiella oxytoca": None,
                                "Bacillus spp, pumilus": None,
                                "Comamonas testosteroni": None,
                                "Streptococcus anginosus": None,
                                "Cellulosimicrobium cellulans/funkeil/marinum": None,
                                "Lactococcus lactis": None,
                                "Staphylococcus aureus": None,
                                "Streptococcus salivarius": None,
                                "Streptococcus parasanguinis": None,
                                "Pseudomonas mendocina ": None,
                                "Actinomyces turicensis": None,
                                "Lysinibacillus spp": None,
                                "Streptococcus mutans": None,
                                "Streptococcus australis": None,
                                "Enterobacter asburiae": None,
                                "Pseudomonas stutzeri": None,
                                "Kocuria spp, salsicia, rhizophila, kristinae": None,
                                "Rothia dentocariosa/mucliaginosa": None,
                                "Pantoea spp": None,
                                "Pseudomonas citronellolis": None,
                                "Streptococcus mitis/oralis/cristatus": None,
                                "Staphylococcus haemolyticus": None,
                                "Serratia marcescens": None,
                                "Staphylococcus lugdunensis": None,
                                "Staphylococcus epidermidis": None,
                                "Actinomyces neuii": None,
                                "Streptomyces": None,
                                "Bacillus licheniformis": None,
                                "Corynebacterium falsenii": None,

                                # Dysbiotic bacteria — no ranges
                                "Morganellla morganii": None,
                                "Providencia stuartii": None,
                                "Klebsiella pneumonia": None,
                                "Providencia rettgeri": None,
                                "Enterobacter cloacae": None,
                                "Pseudomonas aeruginosa": None,
                                "Bacillus cereus": None,
                                "Citrobacter freundii complex": None,
                                "Arcobacter butzieri": None,

                                # Yeast — no ranges
                                "Candida parapsilosis": None,
                                "Rhodotorula glutinis/mucilaginosa": None,
                                "Candida krusei/inconspicua/lambica": None,
                                "Saccharomyces cerevisiae/boulardi": None,
                                "Geotrichum spp.": None,
                                "Candida pararugosa": None,
                                "Candida lusitaniae": None,
                                "Candida zeylanoides": None,
                                "Trichosporon asahii": None,
                                "Trichosporon ovoides": None,
                                "Trichosporon montevideense": None,
                                "Exophiala dermatitidis": None,
                                "Cryptococcus spp, not neoformans": None,
                                "Cryptococcus diffuens": None,
                                "Candida orthopsilosis": None,

                                # Gut Inflammation
                                "Stool Calprotectin UCSD": "<50",
                                "Stool Calprotectin YFH": "<80",
                                "Stool Lysozyme": "<500",
                                "Stool Lactoferrin": "<7.3",
                                "Stool SIgA": "30-275",

                                # Short Chain Fatty Acids
                                "% SCFA Acetate": "50 to 72",
                                "% SCFA Propionate": "11 to 25",
                                "% SCFA Valerate": "1 to 5",
                                "% SCFA Butyrate": "11 to 32",
                                "Total SCFA": "5 to 16",
                                "Butyrate": "0.8-4"
}

In [48]:
# Drop Columns with Low, High, Average,	Number	
df_longBioDairyLS = df_BioDairyLS.drop(columns=['Low', 'High', 'Average', 'Number'])

# Collect columns
cols = list(df_longBioDairyLS.columns)

# Create columns for biomarker, dates, facility



In [49]:
col_biometricPanel = find_exact("Date collected") or cols[0]  # biomarker/test label
col_units = find_exact("Units")
col_range = find_exact("Good Range")
cols_meta = {c for c in [col_biometricPanel, col_units, col_range] if c is not None}

cols_drop = {"low", "high", "average", "number"}

# Collect date column
rgx_date = re.compile(r"\s*\d{1,2}/\d{1,2}/\d{2,4}\s*")  # must look like M/D/YYYY
col_date = [
                c for c in df_BioDairyLS.columns
                if (c not in cols_meta)
                and (str(c).strip().lower() not in cols_drop)
                and rgx_date.fullmatch(str(c))  # must look like M/D/YYYY
]

col_date = [c for c in col_date if not str(c).lower().startswith("unnamed")]

# Facility column
row_one = df_BioDairyLS.iloc[0]
dict_facilityByDate = {
                        dc: (None if is_blank(row_one.get(dc)) 
                        else str(row_one.get(dc)).strip())
                        for dc in col_date\
}


In [50]:
# Collect Biomarkers and Panel Columns from the Date Collected column

lst_panels = [
    "Vitals",
    "LIPID Panel",
    "Comprehensive Metabolic Panel",
    "Kidney",
    "Glucose/Insulin Panel",
    "Hormone Panel",
    "Cancer Panel",
    "Iron Panel",
    "CBC: Red Blood Cell Details",
    "CBC: White Blood Cell Details",
    "Inflammation Panel",
    "Vitamins, Antioxidants, Amino Acid Panel",
    "Stool tests",
    "Expected/Beneficial flora",
    "Commensal (Imbalanced) flora",
    "Dysbiotic bacteria",
    "Yeast",
    "Gut Inflammation",
    "Short Chain Fatty Acids",
]

# --- pull biomarkers from the "Date collected" column and make clean lists ---

# 0) left column cleaned
labels = [clean_label(x) for x in df_BioDairyLS[col_biometricPanel].tolist()]

# 1) drop blanks/separators
labels_noblank = [x for x in labels if x is not None]

# 2) panels found (in-order, unique) limited to your fixed lst_panels
lst_panels_found = uniq_order([x for x in labels_noblank if x in lst_panels])

labels_noblank, lst_panels_found 


(['Weight',
  'Blood Tests',
  'LIPID Panel',
  '237',
  'HDL (High Density Lipoprotein) "good"',
  'VLDL Cholesterol',
  'IDL Chloresterol',
  'Sum Total Cholesterol',
  'Triglycerides',
  'UCSD LDL Particle # by NMR',
  'UCSD HDL Particle #',
  'UCSD Large VLDL Particle #',
  'UCSD Small LDL Particle #',
  'UCSD Large HDL Particle #',
  'UCSD VLDL Particle Size',
  'UCSD LDL Particle Size',
  'UCSD HDL Particle Size',
  'Real-LDL Size Pattern',
  'TG/HDL',
  'Chol/HDL',
  'Lp(a) Chloresterol',
  'APO A1',
  'APOLIPOPROTEIN-(B100)',
  'Ratio APO B100/AI',
  'Comprehensive Metabolic Panel',
  'Electolytes',
  'Sodium',
  'Potassium',
  'Chloride',
  'Calcium',
  'Phosporus',
  'Magnesium',
  'Zinc',
  'Tot CO2',
  'Bicarbonate',
  'Anion Gap',
  'Liver',
  'Total Protein',
  'Albumin',
  'Globulin',
  'A/G ratio',
  'GGTP',
  'LDH',
  'ALT',
  'AST',
  'AST/ALT ratio',
  'Alkaline phosphatase',
  'Total Bilirubin',
  'Direct Bilirubin',
  'Kidney',
  'Blood Urea Nitrogen (BUN)',
  'Cre

In [51]:
# Extract Biomarkrker value which are the column/row, date, biomarker

# rows that are biomarkers (not panel headers)
idx_biomarker_rows = [
                        i for i, x in enumerate(labels)
                        if (x is not None) and (x in dict_biomarkerToPanels)
]

# build long rows directly (no melt, no wide step)
records = []
for i in idx_biomarker_rows:
    biomarker = labels[i]  # already cleaned
    row = df_BioDairyLS.loc[i, col_date]
    for dc, val in row.items():
        if not is_blank(val):
            records.append({"Date": dc, "Biomarker": biomarker, "Value": val})

df_longBioDairyLS= pd.DataFrame.from_records(records)

# tidy types and order (still long)
df_longBioDairyLS["Date"]  = pd.to_datetime(df_longBioDairyLS["Date"], errors="coerce")
df_longBioDairyLS["Value"] = pd.to_numeric(df_longBioDairyLS["Value"], errors="ignore")
df_longBioDairyLS = df_longBioDairyLS.sort_values(["Date","Biomarker"]).reset_index(drop=True)
df_longBioDairyLS

/var/folders/nf/dp5pbtr55rn8mww76lsrkt800000gn/T/ipykernel_59884/2331884804.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_longBioDairyLS["Value"] = pd.to_numeric(df_longBioDairyLS["Value"], errors="ignore")


,Date,Biomarker,Value
0,1993-05-01,Sum Total Cholesterol,152.0
1,1996-11-27,A/G ratio,1.3
2,1996-11-27,AST,37.0
3,1996-11-27,Albumin,4.2
4,1996-11-27,Alkaline phosphatase,64.0
...,...,...,...
22857,2025-06-13,"Vitamin D, 25-Hydroxy",69
22858,2025-06-13,Vitamin E Alpha-Tocopherol,7.9
22859,2025-06-13,Vitamin E Gamma-Tocopherol,0.6
22860,2025-06-13,WBC-,4.4


In [52]:
# make units, panel, ranges, and facilitie columns by mapping

df_longBioDairyLS["Units"] = df_longBioDairyLS["Biomarker"].map(dict_unitsToBiomarkers)
df_longBioDairyLS["Panel"] = df_longBioDairyLS["Biomarker"].map(dict_biomarkerToPanels)
df_longBioDairyLS["GoodRange"] = df_longBioDairyLS["Biomarker"].map(dict_biomarkersToRanges)
df_longBioDairyLS["Facility"] = df_longBioDairyLS["Date"].map(dict_facilityByDate)
df_longBioDairyLS

,Date,Biomarker,Value,Units,Panel,GoodRange,Facility
0,1993-05-01,Sum Total Cholesterol,152.0,mg/dL,LIPID Panel,<199,NaN
1,1996-11-27,A/G ratio,1.3,None,Comprehensive Metabolic Panel,1.1-2.3,NaN
2,1996-11-27,AST,37.0,U/L,Comprehensive Metabolic Panel,0-40,NaN
3,1996-11-27,Albumin,4.2,g/dL,Comprehensive Metabolic Panel,3.5 to 5.2,NaN
4,1996-11-27,Alkaline phosphatase,64.0,U/L,Comprehensive Metabolic Panel,40 to 120,NaN
...,...,...,...,...,...,...,...
22857,2025-06-13,"Vitamin D, 25-Hydroxy",69,ng/mL,"Vitamins, Antioxidants, Amino Acid Panel",30-80,NaN
22858,2025-06-13,Vitamin E Alpha-Tocopherol,7.9,mg/L,"Vitamins, Antioxidants, Amino Acid Panel",5.5-18,NaN
22859,2025-06-13,Vitamin E Gamma-Tocopherol,0.6,mg/L,"Vitamins, Antioxidants, Amino Acid Panel",0-6,NaN
22860,2025-06-13,WBC-,4.4,10**3/mL,CBC: White Blood Cell Details,4 to 10,NaN


In [53]:
# 1) rebuild facility dict with normalized datetime keys; skip bad/blank keys
_fixed = {}
for k, v in dict_facilityByDate.items():
    if is_blank(k):
        continue
    ts = pd.to_datetime(str(k).strip(), errors="coerce")
    if pd.isna(ts):
        continue
    _fixed[ts.normalize()] = (None if is_blank(v) else str(v).strip())
dict_facilityByDate = _fixed  # overwrite with safe version

# 2) map facility WITHOUT changing df_longBioDairyLS["Date"]
df_longBioDairyLS["Facility"] = pd.to_datetime(
    df_longBioDairyLS["Date"], errors="coerce"
).dt.normalize().map(dict_facilityByDate)

# quick check
print(df_longBioDairyLS["Facility"].value_counts(dropna=False).head())

Facility
YFH         10688
UCSD/YFH     4853
UCSD         4773
YFH/UCSD     1125
None         1000
Name: count, dtype: int64


In [54]:
df_longBioDairyLS["Facility"].value_counts()

Facility
YFH                 10688
UCSD/YFH             4853
UCSD                 4773
YFH/UCSD             1125
Hood                  167
YFH/Hood               94
Scripps                69
HLI                    54
13, 2025 (Column       37
OMI                     2
Name: count, dtype: int64

In [55]:
df_longBioDairyLS.to_csv(fp_long,index=False)
print(f"Saved {fp_long}")

Saved Data/LS_Biomarkers_Long.csv


# Make Wide Format

In [56]:
# Creat wide dataframe from longBioDairyLS

# Change column labels to biomarker_unit excpet if "%" contained in biomarker
df_longBioDairyLS["Biomarker_Unit"] = df_longBioDairyLS.apply(
                                                                lambda x: f"{x['Biomarker']}_{x['Units']}"
                                                                if "%" not in x["Biomarker"] else x["Biomarker"],
                                                                axis=1
)

# Pivot to make date the rows and biomarkers the columns
df_wideBioDairyLS = df_longBioDairyLS.pivot(
                                            index="Date", 
                                            columns="Biomarker_Unit", 
                                            values="Value"
).reset_index()
df_wideBioDairyLS 

Biomarker_Unit,Date,% SCFA Acetate,% SCFA Butyrate,% SCFA Propionate,% SCFA Valerate,237_mg/dL,A/G ratio_None,ALT_U/L,APO A1_mg/dL,APOLIPOPROTEIN-(B100)_mg/dL,...,Uric Acid_None,VLDL Cholesterol_mg/dL,Vitamin B12_pg/mL,"Vitamin D, 25-Hydroxy_ng/mL",Vitamin E Alpha-Tocopherol_mg/L,Vitamin E Gamma-Tocopherol_mg/L,WBC-_10**3/mL,Weight_lbs,Zinc_µg/dL,pH_None
0,1993-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-11-27,NaN,NaN,NaN,NaN,NaN,1.3,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,8.7,NaN,NaN,NaN
2,1997-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,NaN,NaN,NaN,NaN,NaN,6.3,NaN,NaN,NaN
3,1998-04-15,NaN,NaN,NaN,NaN,NaN,1.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.4,NaN,NaN,NaN
4,1999-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,2025-02-05,47,24,22,7.2,NaN,NaN,22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.7,178,NaN,6.5
328,2025-02-12,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,...,NaN,NaN,NaN,49,NaN,NaN,4,179,NaN,NaN
329,2025-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.5,179.6,NaN,NaN
330,2025-05-20,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN,NaN,...,NaN,NaN,NaN,64,NaN,NaN,4,182.5,NaN,NaN


In [57]:
df_wideBioDairyLS.to_csv(fp_wide, index=False)
print(f"Saved {fp_wide}") 

Saved Data/LS_Biomarkers_Wide.csv
